In [ ]:
import matplotlib.pyplot as plt

import fastf1
import fastf1.plotting
import pandas as pd

fastf1.plotting.setup_mpl(mpl_timedelta_support=False, misc_mpl_mods=False,
                          color_scheme='fastf1')


In [ ]:
fastf1.events.get_event_schedule(year)

In [49]:
YEARS = [2024]

acc = []

for year in YEARS:
  events = fastf1.events.get_event_schedule(year)

  for i, event in events.iterrows():
    if i != 1:
      continue

    if event.is_testing():
      continue

    session = fastf1.get_session(year, event['RoundNumber'], 'Race')
    session.load(telemetry=True)

    for driver in session.drivers:
      df = session.laps.pick_drivers(driver).get_pos_data().add_driver_ahead()

      mask = (df['DriverAhead'].shift() != df['DriverAhead']) & \
            (df['DriverAhead'].notna()) & \
            (df['DriverAhead'].shift().notna())

      first_row_mask = df.index == 0
      if len(df) > 0 and pd.notna(df.loc[0, 'DriverAhead']):
          mask = mask | first_row_mask

      filtered_df = df[mask]

      filtered_df['DriverId'] = driver
      filtered_df['DriverName'] = session.get_driver(driver).FullName
      filtered_df['RoundNumber'] = event['RoundNumber']
      filtered_df['Country'] = event['Country']
      filtered_df['Location'] = event['Location']
      filtered_df['OfficialEventName'] = event['OfficialEventName']
      # filtered_df['EventDate'] = event['EventDate']
      filtered_df['EventName'] = event['EventName']
      filtered_df['EventFormat'] = event['EventFormat']
      
      acc.append(filtered_df)

df = pd.concat(acc, ignore_index=True)
df


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
/var/folders/k2/kvy782_n4wx4tbsj_0rl6wz80000gn/T/ipykernel_238

,Date,Status,X,Y,Z,Source,Time,SessionTime,DriverAhead,DistanceToDriverAhead,DriverId,DriverName,RoundNumber,Country,Location,OfficialEventName,EventName,EventFormat
0,2024-03-02 15:03:42.460,OnTrack,-280.0,3550.0,-157.0,pos,0 days 00:00:00.118000,0 days 01:00:00.029000,,0.000000,1,Max Verstappen,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
1,2024-03-02 15:03:43.040,OnTrack,-280.0,3558.0,-157.0,pos,0 days 00:00:00.698000,0 days 01:00:00.609000,63,0.000000,1,Max Verstappen,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
2,2024-03-02 15:03:43.460,OnTrack,-280.0,3574.0,-157.0,pos,0 days 00:00:01.118000,0 days 01:00:01.029000,4,0.000000,1,Max Verstappen,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
3,2024-03-02 15:03:44.779,OnTrack,-273.0,3744.0,-158.0,pos,0 days 00:00:02.437000,0 days 01:00:02.348000,14,0.077743,1,Max Verstappen,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
4,2024-03-02 15:03:45.119,OnTrack,-270.0,3824.0,-159.0,pos,0 days 00:00:02.777000,0 days 01:00:02.688000,44,0.222222,1,Max Verstappen,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13774,2024-03-02 15:37:31.459,OnTrack,2313.0,2726.0,-151.0,pos,0 days 00:33:49.117000,0 days 01:33:49.028000,27,3818.793610,2,Logan Sargeant,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
13775,2024-03-02 15:48:43.279,OnTrack,7487.0,6609.0,8.0,pos,0 days 00:45:00.937000,0 days 01:45:00.848000,18,3945.877777,2,Logan Sargeant,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
13776,2024-03-02 15:53:39.239,OnTrack,-324.0,2577.0,-158.0,pos,0 days 00:49:56.897000,0 days 01:49:56.808000,77,5383.768337,2,Logan Sargeant,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
13777,2024-03-02 15:53:50.859,OnTrack,593.0,7979.0,-142.0,pos,0 days 00:50:08.517000,0 days 01:50:08.428000,31,5128.959722,2,Logan Sargeant,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional


In [ ]:
print(df.to_json())

In [48]:
import pandas as pd
import numpy as np

# Create a simple DataFrame with the DriverAhead column
data = {
    'Time': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'DriverAhead': ['Hamilton', 'Hamilton', 'Verstappen', 'Verstappen', None, 
                   'Leclerc', 'Leclerc', None, 'Sainz', 'Sainz'],
    'OtherData': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
}
df = pd.DataFrame(data)

# Display the original DataFrame
# print("Original DataFrame:")
# print(df)
# print("\n")

# Original filtering code from your first question
# print("Method 1 (Original - keeps all transitions):")
filtered_df1 = df[df['DriverAhead'].shift() != df['DriverAhead']]
filtered_df1 = pd.concat([df.iloc[:1], filtered_df1])
filtered_df1 = filtered_df1.reset_index(drop=True)
# print(filtered_df1)
# print("\n")

# The new filtering code that only keeps driver-to-driver transitions
# print("Method 2 (Only driver-to-driver transitions):")
mask = (df['DriverAhead'].shift() != df['DriverAhead']) & \
       (df['DriverAhead'].notna()) & \
       (df['DriverAhead'].shift().notna())

# Include the first row if it has a driver ahead
first_row_mask = df.index == 0
if len(df) > 0 and pd.notna(df.loc[0, 'DriverAhead']):
    mask = mask | first_row_mask

# Filter the DataFrame
filtered_df2 = df[mask]
filtered_df2 = filtered_df2.reset_index(drop=True)
# print(filtered_df2)